## Primary & Blooming Model

In [1]:
from keras.models import Sequential, load_model, Model
from keras.layers import Conv2D, Dense, Flatten, Dropout, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, ModelCheckpoint, ReduceLROnPlateau
from keras.applications.vgg16 import VGG16, preprocess_input

import numpy as np
import cPickle as pickle
import pandas as pd
import cv2
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import os
import sys
sys.path.insert(0, '..')
import paths
sys.path.insert(0, '../rainforest')
import data
import data_generators_leo as dgl

Using Theano backend.
Using cuDNN version 5110 on context None
Mapped name None to device cuda0: GeForce GTX 1050 Ti (0000:09:00.0)


### Load training data and labels.

In [2]:
batch_size = 32
size_img = 92
imgs = (size_img,size_img)
#train_csv = data.get_class_data(train=True, label='primary')

#label is blooming

train_csv =np.asarray(data.get_data(train=True)) #everything in onehot
val_csv = np.asarray(data.get_data(train=False)) #everything in onehot

train_data = train_csv[:,0] #image names training
val_data = val_csv[:,0] #image names validation

#train_labels = train_csv[:,2:3] #blooming
#val_labels = val_csv[:,2:3]
train_labels = train_csv[:,11:12] #primary
val_labels = val_csv[:,2:3]

print train_data.shape
print val_data.shape
print train_labels.shape
print val_labels.shape

(28335,)
(12144,)
(28335, 1)
(12144, 1)


In [3]:
dim = len(train_labels)
dimlabel = np.zeros((dim, 2))
for i, j in zip(train_labels, dimlabel):
    j[i[0]] = 1
train_labels = dimlabel

dim = len(val_labels)
vallabel = np.zeros((dim, 2))
for i, j in zip(val_labels, vallabel):
    j[i[0]] = 1
val_labels = vallabel
print train_labels.shape
print val_labels.shape

(28335, 2)
(12144, 2)


### Generators

In [4]:
train_gen = dgl.get_data(train_data, '../data/train-jpg', train_labels, batch_size=batch_size, img_size=imgs, balance_batches=True, augmentation=True, hflip=True, vflip=True, shift_x=5, shift_y=5, rot_range=10)

val_gen = dgl.get_data(val_data, '../data/train-jpg', val_labels, batch_size=batch_size, img_size=imgs, balance_batches=True)

In [5]:
#imgs, labels = train_gen.next()

#for img, label in zip(imgs, labels):
#    print label

### Network Structure

In [6]:

model = Sequential()

model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(3, 92, 92)))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(Dense(2, activation='softmax'))


model.compile(optimizer='Adam', loss='binary_crossentropy',metrics=['accuracy'])
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 90, 90)        448       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 88, 88)        2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 44, 44)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 44, 44)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 42, 42)        4640      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 40, 40)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 20, 20)        0         
__________

### Training Network

In [7]:
steps = len(train_data)/batch_size
steps_val = len(val_data)/batch_size
print steps
print steps_val


csv_logger = CSVLogger('run4_adam.csv')
lr_plateau = ReduceLROnPlateau(monitor='val_loss', patience=1, verbose=1, factor=0.5)
checkpoint = ModelCheckpoint(filepath='/home/pieter/projects/MLIP-Brainforest/bloomingtest/VGG/model.{epoch:02d}-{val_loss:.2f}.hdf5',
                             verbose=1, save_best_only=True)

model.fit_generator(train_gen, steps_per_epoch=steps,
                    epochs=10, verbose=1,
                    callbacks=[csv_logger, lr_plateau, checkpoint],
                    validation_data=val_gen, validation_steps=steps_val)

885
379
Epoch 1/10
885/885 [==============================] - 73s - loss: 0.6903 - acc: 0.5233 - val_loss: 0.6904 - val_acc: 0.6005
Epoch 2/10
885/885 [==============================] - 70s - loss: 0.6932 - acc: 0.5026 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 3/10
884/885 [============================>.] - ETA: 0s - loss: 0.6931 - acc: 0.5000
Epoch 00002: reducing learning rate to 0.000500000023749.
Epoch 00002: val_loss did not improve
885/885 [==============================] - 70s - loss: 0.6931 - acc: 0.5000 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 4/10
884/885 [============================>.] - ETA: 0s - loss: 0.6931 - acc: 0.4997
Epoch 00003: reducing learning rate to 0.000250000011874.
Epoch 00003: val_loss did not improve
885/885 [==============================] - 71s - loss: 0.6931 - acc: 0.4997 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 5/10
884/885 [============================>.] - ETA: 0s - loss: 0.6931 - acc: 0.4999
Epoch 00004: reducing learning rate to 0.000125000005